In [1]:
import pandas as pd
import os, sys
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)

from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    ts_split,
    cont_cat_split,
    dataloader_by_stock,
    get_data,
    TextTransform
)


C:\Users\gilbe\anaconda3\envs\tokyo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
computer_name1 = 'gilbe'
computer_name2 = 'Gilberto-BE'

ROOT_PATH = f'c:/Users/{computer_name1}/Documents/TokyoData'

In [3]:
stock_list = pd.read_csv(f'{ROOT_PATH}/stock_list.csv').drop('Close', axis=1)
print(stock_list.info())
print()
print(stock_list.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4417 entries, 0 to 4416
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SecuritiesCode          4417 non-null   int64  
 1   EffectiveDate           4417 non-null   int64  
 2   Name                    4417 non-null   object 
 3   Section/Products        4417 non-null   object 
 4   NewMarketSegment        3772 non-null   object 
 5   33SectorCode            4417 non-null   object 
 6   33SectorName            4417 non-null   object 
 7   17SectorCode            4417 non-null   object 
 8   17SectorName            4417 non-null   object 
 9   NewIndexSeriesSizeCode  4417 non-null   object 
 10  NewIndexSeriesSize      4417 non-null   object 
 11  TradeDate               4121 non-null   float64
 12  IssuedShares            4121 non-null   float64
 13  MarketCapitalization    4121 non-null   float64
 14  Universe0               4417 non-null   

In [4]:



'/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv'
'/train_files/trades.csv'

train_df = pd.read_csv(f'{ROOT_PATH}/train_files/stock_prices.csv')
# train_df['Date'] = pd.to_datetime(train_df['Date']) 
# train_df.set_index('Date', inplace=True)
# train_df = date_features(train_df)
print(train_df.head(2))

train_options = pd.read_csv(f'{ROOT_PATH}/train_files/options.csv', low_memory=False)
# train_options['Date'] = pd.to_datetime(train_options['Date'])
# train_options.set_index('Date', inplace=True)

train_financials = pd.read_csv(f'{ROOT_PATH}/train_files/financials.csv', low_memory=False)
# train_financials['Date'] = pd.to_datetime(train_financials['Date'])
# train_financials.set_index('Date', inplace=True)

train_trades = pd.read_csv(f'{ROOT_PATH}/train_files/trades.csv', low_memory=False)


print(train_df.head(2))
print(train_df.tail(2))

print(train_df.info())
print('.' * 20)
print()



print('OPTIONS')
print(train_options.head(2))
print(train_options.info())
print('.' * 50)
print()


print('FINANCIALS')
print(train_financials.head(2))
print(train_financials.info())
print('.' * 50)
print()


print('TRADES')
print(train_trades.head(2))
print(train_trades.info())
print('.' * 50)
print()


           RowId        Date  SecuritiesCode    Open    High     Low   Close  \
0  20170104_1301  2017-01-04            1301  2734.0  2755.0  2730.0  2742.0   
1  20170104_1332  2017-01-04            1332   568.0   576.0   563.0   571.0   

    Volume  AdjustmentFactor  ExpectedDividend  SupervisionFlag    Target  
0    31400               1.0               NaN            False  0.000730  
1  2798500               1.0               NaN            False  0.012324  
           RowId        Date  SecuritiesCode    Open    High     Low   Close  \
0  20170104_1301  2017-01-04            1301  2734.0  2755.0  2730.0  2742.0   
1  20170104_1332  2017-01-04            1332   568.0   576.0   563.0   571.0   

    Volume  AdjustmentFactor  ExpectedDividend  SupervisionFlag    Target  
0    31400               1.0               NaN            False  0.000730  
1  2798500               1.0               NaN            False  0.012324  
                 RowId        Date  SecuritiesCode    Open    

# Merging data

### Ideas
Train some stocks from some industries

In [5]:
stock_list[['SecuritiesCode', 'MarketCapitalization']]

,SecuritiesCode,MarketCapitalization
0,1301,3.365911e+10
1,1305,7.621831e+12
2,1306,1.641739e+13
3,1308,7.671945e+12
4,1309,3.216145e+09
...,...,...
4412,9994,2.654474e+10
4413,9995,1.083499e+10
4414,9996,1.361913e+10
4415,9997,6.894633e+10


In [6]:
train_financials[train_financials['SecuritiesCode'] == 1301]

,DisclosureNumber,DateCode,Date,SecuritiesCode,DisclosedDate,DisclosedTime,DisclosedUnixTime,TypeOfDocument,CurrentPeriodEndDate,TypeOfCurrentPeriod,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
2781,2.016113e+13,20170210_1301,2017-02-10,1301.0,2017-02-10,11:30:00,1.486694e+09,3QFinancialStatements_Consolidated_JP,2016-12-31,3Q,...,199.94,NaN,False,True,False,False,False,10928283,425455,10503022
3940,2.017022e+13,20170217_1301,2017-02-17,1301.0,2017-02-17,15:00:00,1.487311e+09,ForecastRevision,2017-03-31,FY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6892,2.017033e+13,20170511_1301,2017-05-11,1301.0,2017-05-11,11:30:00,1.494470e+09,FYFinancialStatements_Consolidated_JP,2017-03-31,FY,...,257.07,NaN,False,True,False,False,False,10928283,425510,10502960
11274,2.017062e+13,20170804_1301,2017-08-04,1301.0,2017-08-04,13:00:00,1.501819e+09,1QFinancialStatements_Consolidated_JP,2017-06-30,1Q,...,257.07,NaN,False,False,False,False,False,10928283,425510,10502773
15877,2.017091e+13,20171106_1301,2017-11-06,1301.0,2017-11-06,11:30:00,1.509935e+09,2QFinancialStatements_Consolidated_JP,2017-09-30,2Q,...,257.07,NaN,False,False,False,False,False,10928283,422090,10503343
21130,2.017122e+13,20180209_1301,2018-02-09,1301.0,2018-02-09,11:30:00,1.518143e+09,3QFinancialStatements_Consolidated_JP,2017-12-31,3Q,...,257.07,NaN,False,False,False,False,False,10928283,411773,10505430
21131,2.018021e+13,20180209_1301,2018-02-09,1301.0,2018-02-09,11:30:00,1.518143e+09,ForecastRevision,2018-03-31,FY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25190,2.018040e+13,20180510_1301,2018-05-10,1301.0,2018-05-10,11:30:00,1.525919e+09,FYFinancialStatements_Consolidated_JP,2018-03-31,FY,...,296.75,NaN,False,False,False,False,False,10928283,144712,10552710
29535,2.018063e+13,20180803_1301,2018-08-03,1301.0,2018-08-03,11:30:00,1.533263e+09,1QFinancialStatements_Consolidated_JP,2018-06-30,1Q,...,296.75,NaN,False,False,False,False,False,10928283,143521,10783962
34000,2.018092e+13,20181105_1301,2018-11-05,1301.0,2018-11-05,11:30:00,1.541385e+09,2QFinancialStatements_Consolidated_JP,2018-09-30,2Q,...,296.75,NaN,False,False,False,False,False,10928283,109062,10801591


In [7]:
merged_data = stock_list.merge(train_df, on='SecuritiesCode')

merged_data = merged_data.merge(train_financials, on=['SecuritiesCode', 'Date'], how='left')
display(merged_data.head())
display(merged_data.info())

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2335601 entries, 0 to 2335600
Data columns (total 69 columns):
 #   Column                                                                        Dtype  
---  ------                                                                        -----  
 0   SecuritiesCode                                                                int64  
 1   EffectiveDate                                                                 int64  
 2   Name                                                                          object 
 3   Section/Products                                                              object 
 4   NewMarketSegment                                                              object 
 5   33SectorCode                                                                  object 
 6   33SectorName                                                                  object 
 7   17SectorCode                                                   

None

In [8]:
print(merged_data[merged_data['SecuritiesCode'] == 1301].shape)
display(merged_data[merged_data['SecuritiesCode'] == 1301])
print(merged_data[merged_data['SecuritiesCode'] == 1301]['NumberOfTreasuryStockAtTheEndOfFiscalYear'].unique()) 

(1204, 69)


,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1200,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1201,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


[nan '425455' '425510' '422090' '411773' '144712' '143521' '109062'
 '109112' '109196' '109203' '187735' '184241' '170699' '170774' '170681'
 '170467']


In [9]:
print(merged_data[merged_data['SecuritiesCode'] == 1301].columns) 

Index(['SecuritiesCode', 'EffectiveDate', 'Name', 'Section/Products',
       'NewMarketSegment', '33SectorCode', '33SectorName', '17SectorCode',
       '17SectorName', 'NewIndexSeriesSizeCode', 'NewIndexSeriesSize',
       'TradeDate', 'IssuedShares', 'MarketCapitalization', 'Universe0',
       'RowId', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume',
       'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag', 'Target',
       'DisclosureNumber', 'DateCode', 'DisclosedDate', 'DisclosedTime',
       'DisclosedUnixTime', 'TypeOfDocument', 'CurrentPeriodEndDate',
       'TypeOfCurrentPeriod', 'CurrentFiscalYearStartDate',
       'CurrentFiscalYearEndDate', 'NetSales', 'OperatingProfit',
       'OrdinaryProfit', 'Profit', 'EarningsPerShare', 'TotalAssets', 'Equity',
       'EquityToAssetRatio', 'BookValuePerShare',
       'ResultDividendPerShare1stQuarter', 'ResultDividendPerShare2ndQuarter',
       'ResultDividendPerShare3rdQuarter',
       'ResultDividendPerShareFiscalYearEnd', 'R

In [10]:
stock_list['Section/Products'].unique()

array(['First Section (Domestic)', 'ETFs/ ETNs',
       'JASDAQ(Standard / Domestic)', 'JASDAQ(Growth/Domestic)',
       'Mothers (Domestic)', 'Second Section(Domestic)',
       'First Section (Foreign)',
       'REIT, Venture Funds, Country Funds and Infrastructure Funds',
       'JASDAQ(Standard / Foreign)', 'Mothers (Foreign)',
       'Second Section(Foreign)'], dtype=object)

In [11]:
stock_list['NewMarketSegment'].unique()

array(['Prime Market', nan, 'Standard Market', 'Growth Market',
       'Prime Market (Foreign Stock)', 'Standard Market (Foreign Stock)',
       'Growth Market (Foreign Stock)'], dtype=object)

In [12]:
stock_list['IssuedShares'].unique()

array([1.09282830e+07, 3.63463562e+09, 7.91771845e+09, ...,
       9.15264000e+06, 9.72444720e+07, 3.42469620e+07])

In [13]:
print(stock_list.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4417 entries, 0 to 4416
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SecuritiesCode          4417 non-null   int64  
 1   EffectiveDate           4417 non-null   int64  
 2   Name                    4417 non-null   object 
 3   Section/Products        4417 non-null   object 
 4   NewMarketSegment        3772 non-null   object 
 5   33SectorCode            4417 non-null   object 
 6   33SectorName            4417 non-null   object 
 7   17SectorCode            4417 non-null   object 
 8   17SectorName            4417 non-null   object 
 9   NewIndexSeriesSizeCode  4417 non-null   object 
 10  NewIndexSeriesSize      4417 non-null   object 
 11  TradeDate               4121 non-null   float64
 12  IssuedShares            4121 non-null   float64
 13  MarketCapitalization    4121 non-null   float64
 14  Universe0               4417 non-null   

In [14]:
print(stock_list.shape)

(4417, 15)


In [15]:
# !jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000
print(stock_list['NewMarketSegment'])
print()
token_list = list(stock_list['NewMarketSegment'])

print('token_list[:20]:')
print(token_list[:10])
txt_trans = TextTransform(token_list)
print(txt_trans.transform()[:20])



0          Prime Market
1                   NaN
2                   NaN
3                   NaN
4                   NaN
             ...       
4412    Standard Market
4413       Prime Market
4414    Standard Market
4415       Prime Market
4416                NaN
Name: NewMarketSegment, Length: 4417, dtype: object

token_list[:20]:
['Prime Market', nan, nan, nan, nan, nan, nan, nan, nan, nan]
[2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2]
